<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/Var_Control_Crudo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Variables de Control**

Aplicamos el método de **variables de control** para reducir la varianza en una estimación Monte Carlo de la integral:

$$ I = \int_0^1 \frac{1}{1+x^2} \, dx. $$

El valor real de esta integral es:

$$ I = \frac{\pi}{4} \approx 0.785398. $$

In [15]:
# Librerías
import numpy as np
import random as rd

In [16]:
# Función
def g(x):
  return 1 / (1 + x**2)

In [17]:
n = 1000
r = 50

En el método crudo generamos números aleatorios $$U_i \sim \text{Unif}(0,1)$$ y evaluamos la función

$$
g(u_i) = \frac{1}{1+u_i^2}.
$$

El estimador base es:

$$X = \frac{1}{n} \sum_{i=1}^{n} g(U_i)$$

Cuando **n** crece, **X** converge al valor real de la integral.

In [18]:
# Método crudo
def crudo(n):
    G = []
    for i in range(n):
        u = rd.random()
        G.append(g(u))
    return np.mean(G), np.var(G), np.std(G), G

El método de **variables de control** busca reducir la varianza del estimador sin cambiar su valor esperado.

La idea es elegir una variable auxiliar **Y** que:
- esté *correlacionada* con **X**, y
- tenga *esperanza conocida* $$\mu_Y = \mathbb{E}(Y)$$

Entonces definimos un nuevo estimador:

$$Z = X + c \,(Y - \mu_Y)$$

---

**Propiedades:**
- $$\mathbb{E}[Z] = \mathbb{E}[X]$$ → el estimador sigue siendo insesgado.  
- La varianza de **Z** depende de **c**:

$$\mathrm{Var}(Z) = \mathrm{Var}(X) + 2c\,\mathrm{Cov}(X,Y) + c^2 \mathrm{Var}(Y$$

Para minimizar esta varianza respecto a \(c\), derivamos y obtenemos el valor óptimo:

$$
c^* = -\frac{\mathrm{Cov}(X,Y)}{\mathrm{Var}(Y)}
$$

y la varianza mínima resultante es:

$$
\mathrm{Var}(Z) = \mathrm{Var}(X) - \frac{\mathrm{Cov}(X,Y)^2}{\mathrm{Var}(Y)}
$$

Por tanto, si **X** y **Y** están correlacionados, la varianza disminuye.


In [19]:
def control_variate(n, r):

    estimaciones_X = []
    estimaciones_Y = []
    estimaciones_Z = []

    for _ in range(r):
        U = [rd.random() for _ in range(n)] # uniformes
        X = np.mean([g(u) for u in U]) # estimador base
        Y = np.mean(U) # variable de control
        estimaciones_X.append(X)
        estimaciones_Y.append(Y)

    # Calculo de medias
    mu_X = np.mean(estimaciones_X)
    mu_Y = np.mean(estimaciones_Y)
    var_Y = np.var(estimaciones_Y)
    cov_XY = np.cov(estimaciones_X, estimaciones_Y, bias=True)[0,1]

    # constante óptima
    c = - cov_XY / var_Y

    # estimaciones ajustadas
    for i in range(r):
        Z = estimaciones_X[i] + c * (estimaciones_Y[i] - 0.5)
        estimaciones_Z.append(Z)

    return {
        "c": c,
        "media_X": np.mean(estimaciones_X),
        "var_X": np.var(estimaciones_X),
        "media_Z": np.mean(estimaciones_Z),
        "var_Z": np.var(estimaciones_Z)
    }

Usamos como variable de control:

$$Y = \frac{1}{n}\sum_{i=1}^{n} U_i, \quad \text{con } \mathbb{E}(Y) = 0.5$$

Tanto **X** como **Y** se calculan con los mismos números aleatorios **U_i**.

El nuevo estimador corregido es:

$$Z = X + c\,(Y - 0.5)$$

---

**Obtención teórica de *c***

Para $$\sim \text{Unif}(0,1)$$, se tiene:

$$\mathbb{E}[U] = \frac{1}{2}, \qquad \mathrm{Var}(U) = \frac{1}{12}.$$

Y también:

$$\mathbb{E}[f(U)] = \int_0^1 \frac{1}{1+u^2}\,du = \frac{\pi}{4},$$

$$\mathbb{E}[U f(U)] = \int_0^1 \frac{u}{1+u^2}\,du = \frac{1}{2}\ln(2).$$

Por lo tanto,

$$\mathrm{Cov}(f(U),U) = \mathbb{E}[U f(U)] - \mathbb{E}[U]\mathbb{E}[f(U)]
= \frac{1}{2}\ln(2) - \frac{\pi}{8}.$$

Y el valor óptimo de \(c\) resulta ser:

$$c = -\frac{\mathrm{Cov}(X,Y)}{\mathrm{Var}(Y)} = -12 \,\mathrm{Cov}(f(U),U)
= -6\ln(2) + \frac{3\pi}{2}$$

Numéricamente:

$$c \approx 0.5535$$

In [20]:
res = control_variate(n, r)

En el código de Python se estiman empíricamente la covarianza y la varianza:



In [21]:
print("Método de Variables de Control")
print("-----------------------------------------------")
print(f"Constante óptima c:          {res['c']:.10f}")
print(f"Media (crudo):               {res['media_X']:.10f}")
print(f"Varianza (crudo):            {res['var_X']:.10f}")
print(f"Media (con control):         {res['media_Z']:.10f}")
print(f"Varianza (con control):      {res['var_Z']:.10f}")
print("-----------------------------------------------")
print("Valor real de la integral:   π/4 =", np.pi/4)

Método de Variables de Control
-----------------------------------------------
Constante óptima c:          0.5533902107
Media (crudo):               0.7848982469
Varianza (crudo):            0.0000265199
Media (con control):         0.7854190070
Varianza (con control):      0.0000003036
-----------------------------------------------
Valor real de la integral:   π/4 = 0.7853981633974483
